In [ ]:
# -*- coding: utf-8 -*-
# ---
# Projeto: Análise de Dados de E-commerce Brasileiro
# Arquivo: notebooks/notebook_analise_ecommerce.py
#
# INSTRUÇÕES: Este código está completo e pronto. 
# Recomenda-se executar as células (blocos # %%) na ordem 1, 2, 4, 3, 5, 6, 7.
# ---

# %%
# ==============================================================================
# 1. CONFIGURAÇÃO E IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configurações de estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
# %%

# %%
# ==============================================================================
# 2. CARREGAMENTO E JUNÇÃO DE DADOS
# ==============================================================================
DATA_PATH = '../data/raw/' 
try:
    # Carregamento dos dados
    orders_df = pd.read_csv(f'{DATA_PATH}FACT_Orders.csv')
    delivery_df = pd.read_csv(f'{DATA_PATH}DIM_Delivery.csv') 
    shopping_df = pd.read_csv(f'{DATA_PATH}DIM_Shopping.csv')
    products_df = pd.read_csv(f'{DATA_PATH}DIM_Products.csv')
    customer_df = pd.read_csv(f'{DATA_PATH}DIM_Customer.csv')

    # Lógica de Junção (Merge) das 5 tabelas
    merged_df = orders_df.merge(delivery_df, on='Id', how='left')
    merged_df = merged_df.merge(shopping_df, on='Id', how='left', suffixes=('_Order', '_Item'))
    merged_df = merged_df.merge(products_df[['Product_Name', 'Category', 'Subcategory']],
                                left_on='Product', right_on='Product_Name', how='left')
    merged_df = merged_df.merge(customer_df[['Id', 'Region', 'State']], on='Id', how='left')
    merged_df.drop(columns=['Product_Name', 'Id'], inplace=True)
    print(f"Dados Carregados e Juntos. Shape inicial: {merged_df.shape}")

except FileNotFoundError as e:
    print(f"ERRO: Arquivo não encontrado. Verifique se o arquivo está no caminho correto: {e}")
    merged_df = pd.DataFrame() 
# %%

# %%
# ==============================================================================
# 4. FEATURE ENGINEERING (Definição de Funções e Novas Colunas)
# IMPORTANTE: Este bloco cria as colunas e funções necessárias para a limpeza no Bloco 3.
# ==============================================================================

# Função para limitar outliers usando o método IQR (Intervalo Interquartil)
def cap_outliers_iqr(df, col):
    """Limita valores extremos de uma coluna usando o limite de 1.5*IQR."""
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    return df[col]

# Função para calcular Intervalo de Confiança (IC 95%) para a Média
def calculate_confidence_interval_mean(data, confidence=0.95):
    """Calcula o Intervalo de Confiança (IC) para a média de uma amostra."""
    n = len(data)
    if n < 2:
        return np.nan, np.nan
    mean = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return mean - h, mean + h

if not merged_df.empty:
    # 4.0 Tratamento de Tipagem e Datas (CRÍTICO para o Lead Time)
    merged_df['Order_Date'] = pd.to_datetime(merged_df['Order_Date'])
    merged_df['Delivery_Forecast_Date'] = pd.to_datetime(merged_df['D_Forecast'])
    merged_df['Delivery_Date'] = pd.to_datetime(merged_df['D_Date'])

    # 4.1 Features de Logística (KPIs)
    merged_df['Lead_Time_Days'] = (merged_df['Delivery_Date'] - merged_df['Order_Date']).dt.days

    merged_df['is_late'] = np.where(
        (merged_df['Delivery_Date'] > merged_df['Delivery_Forecast_Date']) & 
        (merged_df['Status'] == 'Entregue'), 
        1, 
        0
    )

    # 4.2 Features Financeiras
    merged_df['Freight_Take_Rate'] = merged_df['P_Sevice'] / merged_df['Total'] 
    merged_df['is_confirmed'] = (merged_df['Purchase_Status'] == 'Confirmado').astype(int)
    print("Feature Engineering Concluída (Datas, Lead Time, Atraso, Take Rate).")
# %%

# %%
# ==============================================================================
# 3. DATA CLEANING E QUALIDADE (Executar após o Bloco 4)
# ==============================================================================
if not merged_df.empty:
    # 3.2 Tratamento de Valores Faltantes (NA)
    print("\nContagem de Valores Nulos por Coluna antes da remoção:")
    print(merged_df.isnull().sum()[merged_df.isnull().sum() > 0])

    # 3.2.1 Remoção de Linhas com Valores Ausentes Críticos
    cols_to_drop_na = ['Total', 'Delivery_Date', 'Category']
    initial_shape = merged_df.shape[0]
    merged_df.dropna(subset=cols_to_drop_na, inplace=True)
    
    print(f"\nLinhas removidas devido a NAs em colunas chave.")
    print(f"Linhas Removidas: {initial_shape - merged_df.shape[0]}. Novo shape: {merged_df.shape}")
    print("\nVerificação de Nulos restantes (apenas para referência):")
    print(merged_df.isnull().sum()[merged_df.isnull().sum() > 0])


    # 3.3 Tratamento de Outliers 
    print("\nIniciando tratamento de Outliers (IQR)...")
    outlier_cols = ['Total', 'Subtotal', 'Lead_Time_Days']

    for col in outlier_cols:
        merged_df[col] = cap_outliers_iqr(merged_df, col)

    print("Tratamento de Outliers Concluído.")
    
# %%

# %%
# ==============================================================================
# 5. ANÁLISE EXPLORATÓRIA DE DADOS (EDA)
# ==============================================================================

if not merged_df.empty:
    # 5.1 Gráfico 1: Tendência de Vendas (Sazonalidade da Receita)
    merged_df['Order_Month'] = merged_df['Order_Date'].dt.to_period('M')
    monthly_sales = merged_df.groupby('Order_Month')['Total'].sum()
    
    plt.figure(figsize=(12, 6))
    monthly_sales.plot(kind='line', marker='o', color='teal', linewidth=2)
    plt.title('Tendência de Receita Total ao Longo do Tempo (Vendas Mensais)', fontsize=16)
    plt.xlabel('Mês da Compra', fontsize=12)
    plt.ylabel('Receita Total (R$)', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    # 

    # 5.2 Gráfico 2: Distribuição dos Serviços de Entrega
    plt.figure(figsize=(8, 6))
    sns.countplot(y='Services', data=merged_df, order=merged_df['Services'].value_counts().index, palette='viridis')
    plt.title('Distribuição de Pedidos por Tipo de Serviço de Entrega', fontsize=16)
    plt.xlabel('Contagem de Pedidos', fontsize=12)
    plt.ylabel('Serviço de Entrega', fontsize=12)
    plt.tight_layout()
    plt.show()
    # 

    # 5.3 Gráfico 3: Análise Geográfica (Total de Vendas por Região)
    regional_sales = merged_df.groupby('Region')['Total'].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(10, 7))
    sns.barplot(x=regional_sales.index, y=regional_sales.values, palette='plasma')
    plt.title('Receita Total Distribuída por Região', fontsize=16)
    plt.xlabel('Região', fontsize=12)
    plt.ylabel('Receita Total (R$)', fontsize=12)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
    # 

    print("\nAnálise Exploratória (EDA) Concluída. Gráficos exibidos.")
# %%

# %%
# ==============================================================================
# 6. INFERÊNCIA ESTATÍSTICA E CÁLCULO DE KPIS
# O RESULTADO DESTE BLOCO É O RELATÓRIO FINAL.
# ==============================================================================

if not merged_df.empty:
    # 6.1 KPI Financeiro: Ticket Médio e IC 95%
    tm_mean = merged_df['Total'].mean()
    tm_ic_lower, tm_ic_upper = calculate_confidence_interval_mean(merged_df['Total'])

    # 6.2 KPI Logístico: Lead Time Médio e IC 95%
    lt_mean = merged_df['Lead_Time_Days'].mean()
    lt_ic_lower, lt_ic_upper = calculate_confidence_interval_mean(merged_df['Lead_Time_Days'])

    # 6.3 KPI Operacional: Taxa de Atraso
    delivered_df = merged_df[merged_df['Status'] == 'Entregue']
    delivery_rate = delivered_df['is_late'].mean() * 100 # Em porcentagem

    # 6.4 Taxa de Conversão por Método de Pagamento (Funil)
    conversion_by_payment = merged_df.groupby('payment')['is_confirmed'].mean() * 100

    print("\n" + "="*50)
    print("               RESUMO DE KPIS ESTATÍSTICOS")
    print("="*50)

    print("\n[KPI Financeiro] Ticket Médio (TM):")
    print(f"  R$ {tm_mean:.2f}")
    print(f"  Intervalo de Confiança 95%: [R$ {tm_ic_lower:.2f}, R$ {tm_ic_upper:.2f}]")

    print("\n[KPI Logístico] Lead Time Médio (Tempo de Entrega):")
    print(f"  {lt_mean:.2f} dias")
    print(f"  Intervalo de Confiança 95%: [{lt_ic_lower:.2f} dias, {lt_ic_upper:.2f} dias]")

    print("\n[KPI Operacional] Taxa de Atraso (Pedidos Entregues):")
    print(f"  {delivery_rate:.2f}%")

    print("\n[KPI de Vendas] Taxa de Confirmação de Compra por Pagamento:")
    print(conversion_by_payment.sort_values(ascending=False).to_string(float_format="%.2f%%"))
    
    print("\nCálculo de KPIs e Inferência Estatística Concluída.")
# %%

# %%
# ==============================================================================
# 7. EXPORTAÇÃO E CONCLUSÃO
# ==============================================================================

if not merged_df.empty:
    # Exporta o DataFrame limpo e com as novas features para uso futuro
    try:
        EXPORT_PATH = '../data/processed/df_clean.csv'
        merged_df.to_csv(EXPORT_PATH, index=False)
        print(f"\nDataFrame Limpo exportado com sucesso para: {EXPORT_PATH}")
    except Exception as e:
        print(f"\nERRO ao exportar arquivo: {e}")
# %%

